In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options

import pandas as pd

In [111]:
class coordenadas:
    
    ORIGIN = 'https://www.google.com/maps'
    MAPS_SEARCH_BAR ='searchboxinput'
    NOME = 'section-hero-header-title-title'
    CLASS_RESULTADOS_MAPS = "section-result"
    LAT_LANG = 'widget-reveal-card-lat-lng'
    
    def __init__(self):
        data = {}
        self.info = pd.DataFrame(data)
        self.tabela = pd.read_csv('concorrencia0construcao.csv')
        options = Options()
        options.headless = False
        self.driver = webdriver.Firefox(options=options, executable_path=r'./geckodriver')
        self.driver.implicitly_wait(20)
        self.wait = WebDriverWait(self.driver, 100)
        self.dicionarioInfo = {"Nome": "", "Classificação": "", "Latitude": "", "Longitude": ""}
        for index, loja in self.tabela.iterrows():
            resultado = 0
            print(loja['Nome'])
            self.pesquisar(loja['3'])
            resultado = self.posicionar()
            if resultado == 1:
                self.pesquisar(loja['2'])
                resultado = self.posicionar()
            resultado = self.pegarCoordenadas(loja['Nome'], loja['Avaliação'])
            if resultado == 1:
                self.dicionarioInfo['Latitude'] = 0
                self.dicionarioInfo['Longitude'] = 0
                self.info = self.info.append(self.dicionarioInfo, ignore_index=True)
            else:
                self.info = self.info.append(self.dicionarioInfo, ignore_index=True)
        
        self.info.to_csv('coordenadas.csv', encoding='utf-8')
        
        
    def close(self):
        self.driver.close()
        
    def pesquisar(self, loja):
        self.driver.get(self.ORIGIN)
        originBar = self.driver.find_element_by_id(self.MAPS_SEARCH_BAR)
        originBar.send_keys(str(loja))
        originBar.send_keys(Keys.ENTER)
        return 0
    
    def pegarCoordenadas(self, nome, classificacao):
        try:
            el = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, self.LAT_LANG)))
        except:
            print('Erro ao coletar as coordenadas')
            return 1
        finally:
            try:
                el = self.driver.find_element_by_class_name(self.LAT_LANG)
                coordenadas = el.text.split(",")
            except:
                print('Nao achei as coordenadas')
                return 1
            
            self.dicionarioInfo['Nome'] = nome
            self.dicionarioInfo['Classificação'] = classificacao
            self.dicionarioInfo['Latitude'] = coordenadas[0].replace(" ", "")
            self.dicionarioInfo['Longitude'] = coordenadas[1].replace(" ", "")
            print(self.dicionarioInfo['Latitude'] + ", " + self.dicionarioInfo['Longitude'])

            return 0
    
    def posicionar(self):
        try:
            el = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, self.NOME)))
            #found = self.driver.find_elements(By.XPATH, xpath)
        except:
            print("Não encontrado")
            return 1
        finally:
            try:
                el = self.driver.find_element_by_class_name(self.NOME)
            except:
                return 1
            action = webdriver.common.action_chains.ActionChains(self.driver)
            action.move_to_element_with_offset(el, 865, 10)
            action.context_click()
            action.move_to_element_with_offset(el, 885, 90)
            action.click()
            action.perform()
            return 0
    
    def listarAchados(self):
        try:
            self.wait.until(EC.element_to_be_clickable((By.CLASS_NAME, self.CLASS_RESULTADOS_MAPS)))
        except TimeoutException:
            print("Não encontrado")
        lista = self.driver.find_elements_by_class_name(self.CLASS_RESULTADOS_MAPS)
        #print(len(lista))
        
        for loja in range(len(lista)):
            try:
                WebDriverWait(self.driver, 30).until(EC.element_to_be_clickable((By.CLASS_NAME, self.CLASS_RESULTADOS_MAPS)))
            except:
                print('ERRO 01')
            finally:
                lojas = self.driver.find_elements_by_class_name(self.CLASS_RESULTADOS_MAPS)
                #print(len(lojas))
                #print(loja)
                lojas[loja].click()
                return

        return True
    

In [112]:
teste = coordenadas()

Espaço Smart
-23.299955, -51.184416
Depósito Roseira - Construção e Acabamentos
-23.353330, -51.137791
Depósito São Marcos
-23.301205, -51.169041
Depósito Alvorada - Materiais de Construção
-23.258204, -51.144291
Gol Materiais Para Construção
-23.344830, -51.188916
Depósito de Materiais Casa & Construção
-23.351455, -51.164041
Leroy Merlin Londrina
-23.312455, -51.143541
Todimo
-23.258079, -51.163416
Papaes Comércio Materiais de Construção
-23.309580, -51.155291
Felix Acabamentos e Material Para Construção
-23.310830, -51.195916
Depósito Igapó Materiais p/ Construção
Erro ao coletar as coordenadas
Nao achei as coordenadas
Depósito Aeroporto Materiais para Construção
-23.323705, -51.137041
Todimo
-23.300455, -51.184791
Depósito LC
-23.294580, -51.171416
Dr. Tem Tudo Casa e Construção
-23.320955, -51.189666
Aqui K J R Pisos e Materiais de Construção
Não encontrado
-23.301580, -51.174541
Depósito Eliane Materiais de Construção
-23.367330, -51.141666
Serra Morena Materiais para Construção


In [ ]:
tabela = pd.read_csv('concorrencia0construcao.csv')

In [69]:
tabela.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  206 non-null    int64  
 1   0           0 non-null      float64
 2   1           25 non-null     object 
 3   2           206 non-null    object 
 4   3           203 non-null    object 
 5   4           205 non-null    object 
 6   5           201 non-null    object 
 7   6           81 non-null     object 
 8   Avaliação   181 non-null    object 
 9   Nome        206 non-null    object 
 10  Ranking     181 non-null    object 
 11  7           14 non-null     object 
 12  8           1 non-null      object 
dtypes: float64(1), int64(1), object(11)
memory usage: 21.0+ KB


In [71]:
tabela['2']

0      A, Av. Tiradentes, 1130 - Shangri-lá, Londrina...
1      Av. Europa, 962 - Jd Piza, Londrina - PR, 8604...
2      R. Fortaleza, 271 - Agari, Londrina - PR, 8602...
3      Av. Saul Elkind, 731 - Conj. Semiramis Barros ...
4      Rod. Mabio Gonçalves Palhano, 955 - Gleba Faze...
                             ...                        
201    Av. Winston Churchil, 1360 - Coliseu, Londrina...
202    Av. Dez de Dezembro, 7355 - Lago Igapó, Londri...
203    Av. Waldemar Spranger, 1235 - Tucano, Londrina...
204                                    Adicionar website
205                                londrinatintas.com.br
Name: 2, Length: 206, dtype: object